# Introducción a Python y UDF (*Unified Data Format*)

A manera de introducción de Python, vamos a ver un ejemplo práctico de código utilizado para convertir automáticamente los datos de tomografía eléctrica, que están en Excel, a un formato en archivo de texto que pueda leer PyGIMLi, la librería que vamos a usar para procesar datos eléctricos.

Este formato es el UDF, y está diseñado para hacer que los datos eléctricos sean portables con otras librerías o lenguajes de programación. [Aquí](http://resistivity.net/bert/data_format.html) pueden leer una explicación detallada del formato. 

Los datos de tomografía eléctrica, cuando se toman en campo, tienen un formato así:

![](p1.PNG)

Mientras que un archivo en UDF luce así:

![](p2.PNG)

Si estamos trabajando con varias tomografías, queremos automatizar el proceso para no tener que hacer manualmente la conversión para cada set de datos. Entonces, ¿cómo lo hacemos?

Lo primero que debemos hacer cada vez que usamos Python es importar las librerías que vamos a usar (una vez están instaladas, en el caso de las prepas, todo lo que vamos a necesitar ya está instalado).

In [1]:
import pandas as pd

[Pandas](https://pandas.pydata.org/) es una librería básica para el manejo de datos en Python. Básicamente permite almacenarlos en dos estructuras: 
- Series: un arreglo de una dimensión que puede guardar cualquier tipo de dato.
- DataFrame: un arreglo de dos dimensiones que guarda los datos como una tabla con columnas y filas.

In [2]:
df = pd.read_excel('electric_data _modified.xlsx',sheet_name=0,index_col=None)
data_number = len(df)

Aquí estamos leyendo el archivo Excel "electric_data _modified.xlsx" donde están los datos eléctricos en un DataFrame, llamado ``df`` con la función `pd.read_excel`. Los argumentos de esta función especifican detalles sobre el archivo con el que queremos trabajar, en este caso ``sheet_name`` corresponde a la hoja del archivo que queremos leer.

Por otro lado, ``len(df)`` es una función que nos devuelve la longitud del DataFrame, dato que vamos a utilizar más adelante, por lo que lo almacenamos en una variable llamada ``data_number``.

In [3]:
print(df.head())

   n  A  B   M   N          ΔV           ρ̥   SD         I          SP  \
0  1  0  5  10  15  193.250000  1105.000000  0.0  5.800000 -131.000000   
1  2  0  5  15  20   49.340000  3171.000000  1.4  5.800000   -9.000000   
2  3  0  5  20  25   14.300000  2298.000000  1.2  5.800000  100.000000   
3  4  0  5  25  30    6.200000  1995.000000  1.7  5.800000  258.000000   
4  5  0  5  30  35    2.783333  1554.666667  5.6  5.866667   28.666667   

         Observaciones  
0                  NaN  
1                  NaN  
2                  NaN  
3                  NaN  
4  promedio por sd>1.5  


Con la función ``df.head()`` podemos ver las primeras filas de nuestro DataFrame. Vemos que el parámetro "n" y "Observaciones" no los necesitamos, entonces podemos filtrarlo de nuestro DataFrame, para ello, usamos una función para filtrar y la almacenamos en una nueva variable llamada ``labels_todrop`` y luego aplicamos el filtro al DataFrame:

In [4]:
labels_todrop = ['n', 'Observaciones']
filter = df.filter(labels_todrop)
df.drop(filter, inplace=True, axis=1)

print(df.head())

   A  B   M   N          ΔV           ρ̥   SD         I          SP
0  0  5  10  15  193.250000  1105.000000  0.0  5.800000 -131.000000
1  0  5  15  20   49.340000  3171.000000  1.4  5.800000   -9.000000
2  0  5  20  25   14.300000  2298.000000  1.2  5.800000  100.000000
3  0  5  25  30    6.200000  1995.000000  1.7  5.800000  258.000000
4  0  5  30  35    2.783333  1554.666667  5.6  5.866667   28.666667


Ahora vamos a guardar nombres de columnas de interés en una lista llamada ``column_names``. También, definimos el valor ``electrode_sep``, la separación de electrodos, que es el primer valor de la columna A menos el primero de la columna B. Con la función ``.apply(lambda x:)`` aplicamos esta operación a cada columna, para llevarlo al formato que queremos.

In [5]:
column_names = ['A','B','N','M']

electrode_sep = df.at[1,'B'] - df.at[1,'A']

df[column_names]=df[column_names].apply(lambda x: (x/electrode_sep)+1).astype(int)

Ahora vamos a leer la topografía, que es un archivo con alturas y número de electrodo correspondiente a esa altura.

In [6]:
topo = pd.read_excel('position_xz.xlsx',sheet_name=0)
print(topo)


     x     z
0    1  2855
1    2  2853
2    3  2854
3    4  2853
4    5  2852
5    6  2852
6    7  2851
7    8  2852
8    9  2851
9   10  2850
10  11  2850
11  12  2850
12  13  2850
13  14  2850
14  15  2849
15  16  2849
16  17  2849


Ahora nos gustaría convertir el valor de índice de electrodo a su posición relativa a la separación de electrodos, comenzando en cero, y convertir la altura a profundidad. Para ello, volvemos a usar una función ``lambda x``, esta es especialmente útil para sustituir valores en la misma columna.

In [7]:
topo['x']=topo['x'].apply(lambda x: (((x%(x+1))*electrode_sep)-electrode_sep)).astype(float)
topo['z']=topo['z'] - topo['z'].max()
print(topo)

       x  z
0    0.0  0
1    5.0 -2
2   10.0 -1
3   15.0 -2
4   20.0 -3
5   25.0 -3
6   30.0 -4
7   35.0 -3
8   40.0 -4
9   45.0 -5
10  50.0 -5
11  55.0 -5
12  60.0 -5
13  65.0 -5
14  70.0 -6
15  75.0 -6
16  80.0 -6


Oservamos que el número de filas es el total de electrodos, guardamos esto en una variable "n".

In [8]:
n = len(topo)
print(n)

17


Ahora tenemos que hacer *marramucias* para convertir estos datos en el formato que queremos. Para ello, y mantener el orden de los datos, vamos a utilizar diccionarios.

Primero, creamos "header", un diccionario que será el encabezado del archivo, luego lo convertimos en un DataFrame.

In [9]:
header = [{n:'#x', '#number of electrodes':'z'}]
header = pd.DataFrame(header)
print(header)

   17 #number of electrodes
0  #x                     z


Ahora creamos el encabezado que marca el final de la topografía y el inicio de los datos de resistividad, y lo convertimos en DataFrame.

In [10]:
middle = [{data_number : '#a b m n u','#Number of data': 'rhoa sd i sp'}]
middle = pd.DataFrame(middle)
print(middle)

          105 #Number of data
0  #a b m n u    rhoa sd i sp


Ahora convertimos todos estos DataFrame a un archivo CSV con la función ``.to_csv``, donde el argumento ``mode`` es esencial para crear el archivo adecuadamente: ``mode = w`` significa que se crea el archivo "data_UDF.csv", mientras que ``mode = a`` indica que todos los demás subarchivos se están "pegando" al archivo "data_UDF.csv".

In [11]:
header.to_csv('data_UDF.csv', mode = 'w', index = False,sep='\t')
topo.to_csv('data_UDF.csv', mode = 'a', index = False, header=False,sep='\t')
middle.to_csv('data_UDF.csv', mode = 'a', index = False,sep='\t')
df.to_csv('data_UDF.csv', index=False, mode = 'a', header=False, sep='\t')